In [1]:
from keras.applications.vgg16 import VGG16
import keras
from keras.models import Model
from keras.layers import Dense,GlobalAveragePooling2D
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping
from pickle import dump

Using TensorFlow backend.
/home/vmuser/.conda/envs/snake/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vmuser/.conda/envs/snake/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vmuser/.conda/envs/snake/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vmuser/.conda/envs/snake/l

In [2]:
vggmodel = VGG16(weights='imagenet', include_top=False)

In [3]:
trdata = ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_input)

traindata = trdata.flow_from_directory(directory="datasets/train",color_mode='rgb',batch_size= 32)
# trdata.fit(traindata)

tsdata = ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_input)

testdata = tsdata.flow_from_directory(directory="datasets/val",color_mode='rgb',batch_size= 32)

Found 65918 images belonging to 45 classes.
Found 16499 images belonging to 45 classes.


In [4]:
# traindata.classes.

In [5]:
# vggmodel.summary()

num_class = np.unique(testdata.classes).shape[0]


x=vggmodel.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3

preds = Dense(num_class, activation="softmax")(x)
model_final = Model(inputs = vggmodel.input, outputs = preds)




for layer in model_final.layers[:19]:
    layer.trainable=False
for layer in model_final.layers[19:]:
    layer.trainable=True
    
    
model_final.compile(loss = "categorical_crossentropy", 
                    optimizer = 'Adam', metrics=["accuracy"])


checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=1,
                             save_best_only=True, save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=40, verbose=1, mode='auto')


hist = model_final.fit_generator(generator= traindata, steps_per_epoch= 2060, epochs= 100,
                          validation_data= testdata, validation_steps=0, callbacks=[checkpoint,early])


with open('History', 'wb') as handle: # saving the history of the model
    dump(hist.history, handle)

model_final.save_weights("vgg16_1.h5")


Epoch 1/100
 534/2060 [======>.......................] - ETA: 14:04 - loss: 3.2604 - accuracy: 0.1728

KeyboardInterrupt: 

In [ ]:
# model_final.summary()

In [ ]:
# vggmodel.summary()

In [ ]:
num_class = np.unique(testdata.classes).shape[0]